In [1]:
%config IPCompleter.greedy=True

import numpy as np
import matplotlib.pyplot as plt
import sklearn.preprocessing, sklearn.datasets, sklearn.model_selection

At the end of the previous notebook, I briefly showed one-vs-rest classification technique and normalization of the distribution. We may put this normalization after the sigmoid activation functions - that exactly, what softmax is suppose to do.

# Softmax

Formally, we may define the softmax as follow:

$$
softmax(\pmb{x})_i = \frac{e^{x_i}}{\sum_k e^{x_k}}
$$

In other word, we apply exponential to each input of the softmax and then normalize the probability distribution.

Note that softmax is just a generalization of the sigmoid activation.

$$
softmax(x, 0) = \frac{e^x}{e^x + e^0} = \frac{e^x}{e^x + 1} \cdot \frac{e^{-x}}{e^{-x}} = \frac{e^{x-x}}{e^{x-x} + e^{-x}} = \frac{e^0}{e^0 + e^{-x}} = \frac{1}{1+e^{-x}} = \sigma(x)
$$

Now, when we have the formula, it is easy to implement the sigmoid in numpy.

In [2]:
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x))

But why to bother with sigmoid and not just normalize the outputs of sigmoids? It turns out, we may get better numerical properties if we join sigmoids and normalization into one "layer". Thanks to the following equation.

$$
softmax(\pmb{x}+c)_i=\frac{e^{x_i+c}}{\sum_k e^{x_k+c}} = \frac{e^c}{e^c} \cdot \frac{e^{x_i}}{\sum_k e^{x_k}} = softmax(\pmb{x})_i
$$

The danger in the softmax are the exponents, when the divisor can become too big for float values and as a result become infinity. However, we may set $c=max(\pmb{x})$ and as all the scalars are negative or zero, there would be no overflow. Cases, where the divisor become zero are much more unlikely and even if some scalars are so small, that they are round to zero because of the float precision, it doesn't matter for use to distinguish between their actual value and zero. 

This way, we have our modified softmax function.

In [3]:
def softmax(x):
    x = x - np.max(x)
    return np.exp(x) / np.sum(np.exp(x))

## Gradient

Before we move on, we need to know how to compute gradient. That is not that easy as with softmax, as we have multiple inputs and multiple outputs. You may try it by hand, if you wish.

$$
\frac{\partial softmax(\pmb{x})_i}{\partial x_j} = \frac{\partial \frac{e^{x_i}}{\sum_k e^{x_k}}}{\partial x_j}
$$

There are three indices! Moreover, this is derivative only in respect to one input variable. We need to compute gradient of every output in respect to every input. I don't want to dig too much into the math, you can see [[1]](#Bibliography) if you wanna know more. In the end, the gradient pops out, so we may implement it.

$$
\frac{\partial softmax(\pmb{x})_i}{\partial x_j} = 
\begin{cases}
    softmax(\pmb{x})_j - softmax(\pmb{x})_j \cdot softmax(\pmb{x})_i & \text{if } i = j \\
    0 - softmax(\pmb{x})_j \cdot softmax(\pmb{x})_i & \text{if } i \neq j \\
\end{cases}
$$

You may notice something strange - the gradient is table. Remember, we compute gradient of each output in respect to each input. That need's to be a table. But we want only gradients in respect to the inputs - we need to sum the gradients over the outputs, the same way, as we sum gradients of multiple examples. I won't show the implementation just yet, as I believe it would be more helpfull to view the code and the explanation in bigger picture.

I mentioned **layer** a while ago. The fact is, the softmax is really a layer, how we understand them in the context of neural networks. We will implement our first simple neural networks in the very next notebook. It tooks us eight notebooks, but we finally have enough knowledge to do that. However, we need some refactoring of the code before we can do that.

# Refactoring

Let's start with the **layer** implementation itself. Maybe I should tell first, what the layer is. During the neural network training (we may thing about the logistic regression as about a simple neural network) we are exploiting the chain rule. You may remember from the school the chain rule.

$$
\frac{\partial f(g(h(x)))}{\partial x} = \frac{\partial f(g(h(x)))}{\partial g(h(x))} \cdot \frac{\partial g(h(x))}{\partial h(x)} \cdot \frac{\partial h(x)}{\partial x}
$$

This is not different from our model, remember that loss of logistic regression was written as:

$$
\mathcal{L}(\sigma(\pmb{x}\pmb{w})
$$

And we need to update the weights.

$$
\frac{\mathcal{L}}{\partial \pmb{w}} = \frac{\partial \mathcal{L}}{\partial \sigma} \cdot \frac{\partial \sigma}{\partial \pmb{x}\pmb{w}} \cdot \frac{\partial \pmb{x}\pmb{w}}{\partial \pmb{w}}
$$

Each part of the gradient will be one layer. We will use the same layers for neural nerworks later on. We already implemented some - for example the loss function.

In [2]:
class CategoricalCrossEntropyLoss:
    def __call__(self, target, predicted):
        indices = np.arange(len(target))
        return -np.log(np.maximum(predicted[indices,target], 1e-15))
    
    def gradient(self, target, predicted):
        grad = np.zeros((len(target), 10))
        indices = np.arange(len(target))
        grad[indices,target] = -1 / predicted[indices,target]
        return grad

Notice, that the layer accepts output of the previous layer (in this case in shape `(batchsize, classes)`) and it returns it's gradient in respect to it's input - again in the shape `(batchsize,classes`).

We may use the same approach - let's now implement the sigmoid layer. It takes inputs and apply sigmoid activation function on each of it. It is simple to implement it right away, but there is one problem. If you carefully look at the chain rule, the gradients are multiplied by the gradients of the following layer. In other word, by computing gradient of the sigmoid, we need to compute this:

$$
\frac{\mathcal{L}}{\partial \pmb{w}} = \frac{\partial \mathcal{L}}{\partial \sigma} \cdot \frac{\partial \sigma}{\partial \pmb{x}\pmb{w}}
$$

As a result, the gradient function doesn't accept only layer's input, but gradient from the following layer as well. Now we may implement the sigmoid activation layer.

In [3]:
class SigmoidLayer:
    def __call__(self, inputs):
        return 1 / (1 + np.exp(-inputs))
    
    def gradient(self, inputs, gradients):
        outputs = self(inputs)
        my_gradient = outputs * (1 - outputs)
        return my_gradient * gradients

Now let's look at the softmax layer. This case is a bit complicated, as the gradient is not a vector, but a table. In general, most layers that have multiple inputs and multiple outputs will generate gradient table (note that for each example), as it needs to compute gradient of each output in respect to each input. The sigmoid activation layer, and activation layers in general, are exceptions. Each output corresponds to one input and as such the inputs doesn't interfere. You may look at it from the other side, the gradient of output $i$ in respect to input $j$ is zero if $i \neq j$. That results in table full of zeros except diagonals.

But let's return back to the softmax layer. We already implemented it above, but only for a single example. We need to generalize it for batch of examples. Only thing we need to make sure about is to divide inputs in the correct axis, the generalization is simple.

In [53]:
def batch_softmax(inputs):
    inputs = inputs - np.max(inputs)
    return np.exp(inputs) / np.sum(np.exp(inputs), axis=-1)[:,np.newaxis]

Now let's look at the gradient, the formula is above. It would be simpler to rewrite it in the matrix form, so we may implement it in the numpy. Notice that the only difference is on the diagonal (when $i - j$) where we add additional term. In a matrix notation it results in following equation (as you may see in [[1]](#Bibliography) as well):

$$
\frac{\partial softmax(\pmb{x})}{\partial \pmb{x}} = 
\begin{pmatrix}
o_1 & 0 & \dots & 0 \\ 
0 & o_2 & \dots & 0 \\
\vdots & \vdots & \ddots & \vdots \\
0 & 0 & \dots & o_k
\end{pmatrix}
-
\begin{pmatrix}
o_1 o_1 & o_1 o_2 & \dots & o_1 o_k \\ 
o_2 o_1 & o_2 o_2 o_2 & \dots & o_2 o_k \\
\vdots & \vdots & \ddots & \vdots \\
o_k o_1 & o_k o_2 & \dots & o_k o_k
\end{pmatrix}
$$

The vector $\pmb{o}$ is the output of softmax $softmax(\pmb{x})=\pmb{o}$. Notice the symetry of the matrices, we will exploit it in a minute. If we have variable `outputs` thats contains the output of the softmax function, we may implement the matrix notations to the numpy very simply.

In [55]:
def softmax_gradient(inputs):
    outputs = softmax(inputs)
    return np.diag(outputs) - outputs @ outputs

There are three things we need to deal before we may continue. First problem is the matrix of the gradient - remember we want the gradient in respect to the inputs of the softmax, so we may pass it to the previous layer (remember the chain rule I showed you before). Now we have a matrix - gradient of each output in respect to each input. All we need to is to sum up the gradients over the outputs. As a result, we get a vector, that represents the "acumulated" gradient in respect to each input. I may formulate it in other way around - we receive gradient of how much we want to shift each output. Each output tells us, how much it wants to shift it's input so the output is shifted the corrrect way (and therefore we get the matrix). In the end, we want to satisfy all the outputs - we sum the effect of each output on the inputs and as a result, we get the vector that represents the gradient in respect to the inputs.

In [56]:
def softmax_gradient(inputs):
    outputs = softmax(inputs)
    return np.sum(np.diag(outputs) - outputs @ outputs, axis=1)

Second problem that I didn't account for yet is the gradient from the next layer. Once again, remember the chain rule. The gradient of the softmax need's to be multiplied with the gradient of the following layer (for example the cross entropy loss). We just need to make sure, we multiply the gradient along a correct axis. We think about the matrix above as a gradient of outputs in respect to the inputs, in other word at index $i,j$ is gradient of output $j$ in respect to input $i$, we need to multiply each row and then sum the gradients over columns. As I pointed out, the matrix is in this case symetric and as such doesn't matter in which way we multiply it - we just need to make sure, we multiply it in different axis that we are summing it later.

In [57]:
def softmax_gradient(inputs, gradients):
    outputs = softmax(inputs)
    my_gradient = np.diag(outputs) - outputs @ outputs
    return np.sum(gradients[np.newaxis,:] * my_gradient, axis=1)

Finally, the third and the last problem - we didn't count with the batches. The code above won't work for multiple examples, as with softmax, we need to generalize it. We can't rely on vector and matrix multiplication, as we have higher dimensional objects. The code is bellow and it get's a bit nasty, I add comments about the shapes of numpy arrays, so you may better understand it. I wish I could write it more readable, but this is as far as I could get.

There are two more attributes - `params` and `grads`. We don't need them yet and I will talk about them in a while, for now just ignore them.

In [61]:
class SoftmaxLayer:
    def __init__(self):
        self.params = []
        self.grads = []
    
    def __call__(self, inputs):
        inputs = inputs - np.max(inputs)
        return np.exp(inputs) / np.sum(np.exp(inputs), axis=-1)[:,np.newaxis]
    
    def gradient(self, inputs, gradients):
        outputs = self(inputs)  # examples, classes
        examples, classes = outputs.shape
        diag = np.zeros((examples, classes, classes))  # examples, classes, classes
        diag[:, np.arange(classes), np.arange(classes)] = outputs # set the diagonal of each example
        my_gradient = diag - outputs[:,:,np.newaxis] * outputs[:,np.newaxis,:]  # examples, classes, classes
        return np.sum(gradients[:,np.newaxis,:] * my_gradient, axis=2) # examples, classes

# Dense layer

Now it is time for our last layer. If you remember, the logistic regression had vector of weights, that was multiplied with the example vector (or matrix when we had batch). For the one-vs-rest approach we had multiple logistic regressions and when we were processing the examples, we need to pass it through every neuron. As turns out, we can join the weights of all the neurons together into the weights matrix.

$$
\mathbb{X}\mathbb{W}
$$

Moreover, we may export the bias explicitly and get rid of the additional $1$ that we needed to pad the examples. As a result, we may express the whole loss of multiclass logistic regression in the following formula.

$$
\text{CrossEntropy}(\text{softmax}(\mathbb{X}\mathbb{W}+\mathbb{b}))
$$

The layer, that is responsible for the linear kombinations (the $\mathbb{X}\mathbb{W}+\mathbb{b}$) is called **dense layer** or **fully-connected layer**. That is exactly what we will now focus on.

# Bibliography

- \[1\] The Softmax Function Derivative, Stephen Oman, 17th June 2019, [online](https://aimatters.wordpress.com/2019/06/17/the-softmax-function-derivative/)